In [1]:
from PIL import Image, ImageDraw
from IPython.display import display
import random
import json
import numpy as np


In [2]:
# Each image is made up a series of traits
# The weightings for each trait drive the rarity and add up to 100%

background = ["Blue", "Orange"] 
background_weights = [30, 70]

body = ["giraffe", "tiger", "monkey", "rhino", "elephant", "koala"] 
body_weights = [10, 15, 40, 20, 5, 10]

head = ["giraffe", "tiger", "monkey", "rhino", "elephant", "koala"] 
head_weights = [40, 5, 20, 10, 10, 15]

face = ["giraffe", "tiger", "monkey", "rhino", "elephant", "koala"] 
face_weights = [5, 10, 25, 30, 10, 20]

# Dictionary variable for each trait. 
# Eech trait corresponds to its file name
# Add more shapes and colours as you wish

background_files = {
    "Blue": "blue",
    "Orange": "orange",
}

body_files = {
    "giraffe":"body_giraffe",
    "tiger":"body_tiger",
    "monkey":"body_monkey",
    "rhino":"body_rhino",
    "elephant":"body_elephant",
    "koala":"body_koala",
}

head_files = {
    "giraffe":"head_giraffe",
    "tiger":"head_tiger",
    "monkey":"head_monkey",
    "rhino":"head_rhino",
    "elephant":"head_elephant",
    "koala":"head_koala",
}

face_files = {
    "giraffe":"face_giraffe",
    "tiger":"face_tiger",
    "monkey":"face_monkey",
    "rhino":"face_rhino",
    "elephant":"face_elephant",
    "koala":"face_koala",
}

In [3]:
TOTAL_IMAGES = 100 # Number of random unique images we want to generate ( 2 x 2 x 2 = 8)

all_images = [] 

def create_new_image():

    new_image = {} #

    # For each trait category, select a random trait based on the weightings 
    new_image ["Background"] = random.choices(background, background_weights)[0]
    new_image ["Body"] = random.choices(body, body_weights)[0]
    new_image ["Head"] = random.choices(head, head_weights)[0]
    new_image ["Face"] = random.choices(face, face_weights)[0]

    if new_image in all_images:
        return create_new_image()
    else:
        return new_image


# Generate the unique combinations based on trait weightings
for i in range(TOTAL_IMAGES): 

    new_trait_image = create_new_image()

    all_images.append(new_trait_image)

In [4]:
def all_images_unique(all_images):
    seen = list()
    return not any(i in seen or seen.append(i) for i in all_images)

print("Are all images unique?", all_images_unique(all_images))

Are all images unique? True


In [5]:
i = 0
for item in all_images:
    item["tokenId"] = i
    i = i + 1

In [6]:
print(all_images)

[{'Background': 'Orange', 'Body': 'rhino', 'Head': 'koala', 'Face': 'monkey', 'tokenId': 0}, {'Background': 'Blue', 'Body': 'monkey', 'Head': 'koala', 'Face': 'elephant', 'tokenId': 1}, {'Background': 'Orange', 'Body': 'monkey', 'Head': 'monkey', 'Face': 'rhino', 'tokenId': 2}, {'Background': 'Orange', 'Body': 'rhino', 'Head': 'elephant', 'Face': 'giraffe', 'tokenId': 3}, {'Background': 'Blue', 'Body': 'rhino', 'Head': 'giraffe', 'Face': 'rhino', 'tokenId': 4}, {'Background': 'Orange', 'Body': 'monkey', 'Head': 'rhino', 'Face': 'elephant', 'tokenId': 5}, {'Background': 'Blue', 'Body': 'tiger', 'Head': 'giraffe', 'Face': 'rhino', 'tokenId': 6}, {'Background': 'Blue', 'Body': 'rhino', 'Head': 'koala', 'Face': 'koala', 'tokenId': 7}, {'Background': 'Orange', 'Body': 'monkey', 'Head': 'koala', 'Face': 'monkey', 'tokenId': 8}, {'Background': 'Blue', 'Body': 'monkey', 'Head': 'giraffe', 'Face': 'tiger', 'tokenId': 9}, {'Background': 'Orange', 'Body': 'monkey', 'Head': 'giraffe', 'Face': 'ele

In [7]:
background_count = {}
for item in background:
    background_count[item] = 0

body_count = {}
for item in body:
    body_count[item] = 0

head_count = {}
for item in head:
    head_count[item] = 0
    
face_count = {}
for item in face:
    face_count[item] = 0

for image in all_images:
    background_count[image["Background"]] += 1
    body_count[image["Body"]] += 1
    head_count[image["Head"]] += 1
    face_count[image["Face"]] += 1

print(background_count)
print(body_count)
print(head_count)
print(face_count)

{'Blue': 33, 'Orange': 67}
{'giraffe': 10, 'tiger': 14, 'monkey': 37, 'rhino': 20, 'elephant': 7, 'koala': 12}
{'giraffe': 27, 'tiger': 7, 'monkey': 16, 'rhino': 16, 'elephant': 16, 'koala': 18}
{'giraffe': 6, 'tiger': 12, 'monkey': 17, 'rhino': 31, 'elephant': 11, 'koala': 23}


In [8]:
METADATA_FILE_NAME = './metadata/all-traits.json'; 
with open(METADATA_FILE_NAME, 'w') as outfile:
    json.dump(all_images, outfile, indent=4)

In [9]:
for item in all_images:

    background = Image.open(f'./layers/backgrounds/{background_files[item["Background"]]}.png').convert('RGBA')
    im1 = Image.open(f'./layers/bodies/{body_files[item["Body"]]}.png').convert('RGBA')
    im2 = Image.open(f'./layers/heads/{head_files[item["Head"]]}.png').convert('RGBA')
    im3 = Image.open(f'./layers/faces/{face_files[item["Face"]]}.png').convert('RGBA')

    #Create each composite
    
    # Calculate width to be at the center
    width = (background.width - im1.width) // 2
    # Calculate height to be at the center
    height = (background.height - im1.height) // 2
    
    im1.paste(im2, (width, height), im2)
    im1.paste(im3, (width, height), im3)
    
    background.paste(im1, (width, height), im1)

    #Convert to RGB
    rgb_im = background.convert('RGB')
    file_name = str(item["tokenId"]) + ".png"
    rgb_im.save("./images/" + file_name)

In [10]:
f = open('./metadata/all-traits.json',) 
data = json.load(f)

IMAGES_BASE_URI = "ADD_IMAGES_BASE_URI_HERE"
PROJECT_NAME = "ADD_PROJECT_NAME_HERE"

def getAttribute(key, value):
    return {
        "trait_type": key,
        "value": value
    }
for i in data:
    token_id = i['tokenId']
    token = {
        "image": IMAGES_BASE_URI + str(token_id) + '.png',
        "tokenId": token_id,
        "name": PROJECT_NAME + ' ' + str(token_id),
        "attributes": []
    }
    token["attributes"].append(getAttribute("Background", i["Background"]))
    token["attributes"].append(getAttribute("Body", i["Body"]))
    token["attributes"].append(getAttribute("Head", i["Head"]))
    token["attributes"].append(getAttribute("Face", i["Face"]))

    with open('./metadata/' + str(token_id), 'w') as outfile:
        json.dump(token, outfile, indent=4)
f.close()